In [1]:
#  Copyright (c) 2020. MOOD project
from cleantext import clean
import string
import re
import wikipedia
import time
import os
import random

In [2]:
def remov_pattern():
    remove = string.punctuation
    remove = remove.replace("'", "")  # don't remove hyphens
    pattern = r"[{}]".format(remove)  # create the pattern
    return pattern


In [3]:
def text_clearner(raw_text):
    cleaned = clean(raw_text,lang="fr")
    pattern = remov_pattern()
    cleaned_text = re.sub(pattern, "", cleaned)
    return cleaned_text


In [17]:
# def search_wiki(keywords, number_of_search,nb_page, wiki_pages):
def search_wiki_(keywords, number_of_search,wiki_pages):
    #print('xxxx')
    suggestion = False
    wikipedia.set_lang("fr")
    result_set = wikipedia.search(keywords, number_of_search, suggestion)
    for term in result_set[10:]: # go with 10 first pages
        #print(term)
        try:
            page = wikipedia.page(term, preload=False)
        except wikipedia.DisambiguationError as e:
            # s = wikipedia.random.choice(e.options)
            s = random.choice(e.options)
            page = wikipedia.page(s)
        '''page = wikipedia.page(term, preload=False)'''
        page_title = page.title
        #print(page_title)
        page_content = page.content
        #time.sleep(2)
        wiki_pages[page_title] = page_content
        if not os.path.exists(os.getcwd()+'/'+str(keywords)):
            os.makedirs(os.getcwd()+'/'+str(keywords))
        root = os.getcwd() + '/' + str(keywords)
        fw = open(root + '/' + str(keywords) + '_wiki_corpus.txt', 'a+')
        fw.write("%s\n" % text_clearner(page_content))
        fw.write("\n##########END##########\n")
        fw.close()
        time.sleep(60)
        #print(wiki_pages)
    return wiki_pages
#'/home/rodrique/Bureau/Protocole/corpus_20/'

In [18]:
hydro_keyword = 'agriculture'
nbs = 15
wiki_pages ={}
result_set = search_wiki_(hydro_keyword,nbs,wiki_pages)
result_set

xxxx


{'Herse (agriculture)': "Une herse est un instrument agricole constitué d'un châssis en forme de grille, formée par deux séries de barres, les unes verticales, les autres horizontales, parallèles entre elles et fixées aux points de croisement.\nCet outil est muni de dents courtes et proches les unes des autres qui permettent de travailler la terre en surface pour préparer un lit de semence ou pour le recouvrir. Son passage aplanit le sol, en pulvérisant les mottes, et en enlevant tout ce qui pourrait gêner la germination de la semence.\nPline et d'autres historiens agronomes de la même époque (Caton, Varron, Columelle, Palladius) mentionnent déjà la herse.\n\n\n== Herse étrille ==\nUne herse étrille, est une variante de la herse qui dispose d'un grand nombre de dents tractées (en général par un tracteur). Elle se présente sous la forme d'un châssis sur deux niveaux, les griffes étant souples, elles grattent le sol pour en dégager les plantules.\n\n\n== Calendrier républicain ==\nDans l

# 2 - EXTRACT TERMS FROM "wiki_pages"  USING BIOTEX


# 3 - CHOIX DES TERMES EXPERTS

# 4 - COLLECTE DE DOCUMENTS VIA GOOGLE EN UTILISANT LES TERMES EXPERTS

In [6]:
def pause(logger, minutes=0.5):
    """
		Effectue une pause.

		@type  log: Logger.
		@param log: Fichier de log.

		@type  minutes: Entier.
		@param minutes: Temps en minute à attendre.
	"""
    date = datetime.now().strftime('%d-%m-%Y:%Hh%M')
    current_time = datetime.now() + timedelta(minutes=minutes)

    logger.info("{0} : Nombre limite de requete atteint. Reprise du programme : {1}".format(date, current_time.strftime(
        '%d-%m-%Y:%Hh%M')))

    while datetime.now() < current_time:
        time.sleep(0.5)

In [7]:
#with open("keywords/agriculture.txt") as keywords_file:
#    words = keywords_file.readlines()
#    log.info("Lecture de keywords.txt ... \n")

In [8]:
def generation_requetes(motscles):    
    """
		Génère les requêtes à effectuer.
		@type  motscles: Liste de string.
		@param motscles: Liste des mots clés à rechercher."""
    
    with open(motscles) as keywords_file:
        words = keywords_file.readlines()
        print(words)

    for i in range(0, len(words), 1):
        print(i)
        # motscles_couple = motscles[i].split("+")

        requete = "{0}".format(words[i])

        if not any(requete in s for s in requetes_effectuees):
            yield requete

In [9]:
#kw = "/home/rodrique/Bureau/Jupyter-notebook/Protocole_3M/doc_collect_adapted/keywords/urbanisation_500.txt"
#x = generation_requetes(kw)
#for requete in x:
#    print('###', requete)

In [10]:
import requests
from bs4 import BeautifulSoup
def run_requete(search):
    q = '+'.join(search.split())
    url = 'https://www.google.com/search?q=' + q + '&ie=utf-8&oe=utf-8'
    page = requests.get(url)
    
    #page = requests.get(f"https://www.google.com/search?q={search}")
    soup = BeautifulSoup(page.content, "html5lib")
    links = soup.findAll("a")
    url_list = []
    for link in links :
        print(link)
        link_href = link.get('href')
        if "url?q=" in link_href:# and not "webcache" in link_href:
            print (link.get('href').split("?q=")[1].split("&sa=U")[0])
            url_list.append(link.get('href').split("?q=")[1].split("&sa=U")[0])
            
    return url_list[:-1]


In [11]:
from bs4 import BeautifulSoup

headers_Get = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:49.0) Gecko/20100101 Firefox/49.0',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
        'Accept-Language': 'en-US,en;q=0.5',
        'Accept-Encoding': 'gzip, deflate',
        'DNT': '1',
        'Connection': 'keep-alive',
        'Upgrade-Insecure-Requests': '1'
    }


def google(q):
    s = requests.Session()
    q = '+'.join(q.split())
    url = 'https://www.google.com/search?q=' + q + '&ie=utf-8&oe=utf-8'
    r = s.get(url, headers=headers_Get)

    soup = BeautifulSoup(r.text, "html.parser")
    output = []
    for searchWrapper in soup.find_all('h3', {'class':'r'}): #this line may change in future based on google's web page structure
        url = searchWrapper.find('a')["href"] 
        text = searchWrapper.find('a').text.strip()
        result = {'text': text, 'url': url}
        output.append(result)

    return output

In [23]:
def googleSearch(query):
    with requests.session() as c:
        url = 'https://www.google.co.in'
        query = {'q': query}
        urllink = requests.get(url, params=query)
        print(urllink.url)

